# Introducción

Este proyecto utiliza la **API-FOOTBALL v3**, una API REST con información actualizada e histórica de más de **1100 ligas y copas** (resultados, alineaciones, eventos, clasificaciones y estadísticas).  
El acceso se realiza mediante solicitudes **HTTP autenticadas con API key**, con filtros por **fecha, liga, equipo o temporada**.

En este notebook se documenta y ejecuta el **pipeline ETL orquestado con Prefect**, cuya lógica reside en `scripts/etl_fixtures.py`.  
Se muestra el **setup**, la **ejecución del flow**, y cómo **servir y monitorear** el flujo desde **Prefect Cloud**.

El pipeline persiste datos en un **Data Lake estructurado en capas Delta Lake** (**Bronze → Silver → Gold**) y genera **exportables en CSV y Parquet**.

Además, el flujo puede ejecutarse:
- 🧩 **Manualmente** desde este notebook, para validaciones o cargas controladas.  
- 🔁 **Automáticamente**, programado una vez al día mediante Prefect (cron diario a las 06:00 UTC).


## 1. Configuración de entorno y rutas


In [6]:
# Este bloque asegura que exista la estructura mínima del datalake
# al correr el flujo desde el notebook (modo demo/documentación).
# Nota: en el script etl_fixtures.py las rutas se definen de nuevo
# para la ejecución real por CLI / Prefect.

import os

DATALAKE_ROOT = "data/etl_datalake"
BRONZE_FIXTURES = f"{DATALAKE_ROOT}/bronze/api_football/fixtures"
SILVER_FIXTURES = f"{DATALAKE_ROOT}/silver/api_football/fixtures"
GOLD_FIXTURES   = f"{DATALAKE_ROOT}/gold/api_football/fixtures"
EXPORTS_DIR     = f"{DATALAKE_ROOT}/exports"

for p in [BRONZE_FIXTURES, SILVER_FIXTURES, GOLD_FIXTURES, EXPORTS_DIR, "scripts"]:
    os.makedirs(p, exist_ok=True)

print("Estructura base creada/validada ✅")

Estructura base creada/validada ✅


## 2. Credenciales y configuración


In [7]:
# Se lee la configuración desde pipeline.conf
# Contiene dos secciones:
# [api-credentials] -> API-Football
# [prefect]         -> Prefect Cloud

from configparser import ConfigParser

parser = ConfigParser()
parser.read("pipeline.conf")

# --- API-Football ---
BASE_URL = parser["api-credentials"]["base_url"]
API_KEY  = parser["api-credentials"]["api_key"]

print("Config OK ✅")
print("BASE_URL:", BASE_URL)
print("API_KEY:  (oculto) ****")

Config OK ✅
BASE_URL: https://v3.football.api-sports.io
API_KEY:  (oculto) ****


## 3. Prefect: conexión y conceptos

Prefect permite **orquestar** un pipeline en pasos claros:

- **Flow** → el contenedor principal del proceso ETL.  
- **Task** → cada paso atómico dentro del flow (ej. extracción, transformación, carga).  
- **Serve / Deploy** → maneras de ejecutar y programar flows:  
  - *Serve*: correrlo como servicio local (útil en desarrollo).  
  - *Deploy*: registrarlo en Prefect Cloud para programarlo y monitorearlo.  

El login en Prefect Cloud se realiza *una sola vez desde la terminal*.
Aquí solo se muestra de manera ilustrativa.


In [8]:
# Prefect requiere login en Cloud, pero este paso se hace normalmente
# desde la terminal (una única vez por entorno).
# Aquí solo mostramos cómo se lee la API Key desde pipeline.conf.

prefect_credentials = parser["prefect"]
prefect_api_key = prefect_credentials["api_key"]

print("Prefect API Key cargada (oculta) ****")

# Login en Prefect Cloud (solo a modo ilustrativo; en práctica se hace una vez desde la terminal)
# !prefect cloud login -k {prefect_api_key}

Prefect API Key cargada (oculta) ****


In [9]:
import prefect
print("Prefect versión:", prefect.__version__)

Prefect versión: 2.20.9


## 4. Uso del flujo desde `scripts/etl_fixtures.py`

El flujo ETL está definido en `scripts/etl_fixtures.py`.  
En este notebook es posible:

- **4.1 Opción A — Corrida única (demo one-off):** ejecuta el flow una sola vez para validar la orquestación.  
- **4.2 Opción B — Servir el flow (opcional):** mantiene el flow activo como servicio local.


### 4.1 Opción A — Ejecutar una corrida orquestada (demo one-off)


In [10]:
import importlib

# Importa el script de orquestación
etl = importlib.import_module("scripts.etl_fixtures")

# Ejecuta el flujo ETL de forma local (modo recomendado dentro del Notebook)
etl.etl_parametrizable(endpoints=["fixtures"])

# Nota:
# El mismo flujo también puede ejecutarse desde la terminal:
#     python scripts/etl_fixtures.py
# o dentro del Notebook:
#     !python scripts/etl_fixtures.py
# Todas estas opciones llaman al mismo flow definido en scripts/etl_fixtures.py.

06:29:26.607 | INFO    | prefect.engine - Created flow run 'dangerous-flounder' for flow 'etl-parametrizable'

06:29:26.610 | INFO    | Flow run 'dangerous-flounder' - View at https://app.prefect.cloud/account/1513bf29-3686-40b8-9dbf-c85ba6a6f8c0/workspace/377710aa-6343-48a1-a52d-8fd9be6fbba7/flow-runs/flow-run/0691847f-ba22-7041-8000-e49f95c4769b

06:29:27.339 | INFO    | Flow run 'dangerous-flounder' - Created task run 'extract_data-0' for task 'extract_data'

06:29:27.352 | INFO    | Flow run 'dangerous-flounder' - Executing 'extract_data-0' immediately...

06:29:29.271 | INFO    | Task run 'get_data-fixtures' - Finished in state Completed()

...

06:29:36.037 | INFO    | Flow run 'dangerous-flounder' - Created task run 'transform_to_gold_from_silver-0' for task 'transform_to_gold_from_silver'

06:29:36.037 | INFO    | Flow run 'dangerous-flounder' - Executing 'transform_to_gold_from_silver-0' immediately...

06:29:37.698 | INFO    | Task run 'to_gold-fixtures' - Finished in state Completed()

06:29:38.208 | INFO    | Flow run 'dangerous-flounder' - Finished in state Completed('All states completed.')

[Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `list`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `DataFrame`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `NoneType`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `DataFrame`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `DataFrame`'))]

### 4.2 Opción B — Servir el flow como agente local (opcional)

Esta modalidad permite dejar el flujo **en ejecución continua** y gestionarlo desde **Prefect Cloud** (ejecuciones, logs y programación).  
⚠️ Al activarla, la celda quedará ocupada hasta que la detengas manualmente desde tu entorno (*Interrupt/Stop* del notebook).  
Por defecto, se mantiene **comentada** para evitar que se ejecute automáticamente.  
En caso de habilitarla, el flujo se ejecutará **una vez al día (06:00 UTC)** según la configuración del script `etl_fixtures.py`.

```python
import importlib
etl = importlib.import_module("scripts.etl_fixtures")
# etl.etl_parametrizable.serve(
#     name="ETL-Fixtures",
#     endpoints=["fixtures"],
#     cron="0 6 * * *"  # Una vez al día, 06:00 UTC (ver https://crontab.guru)
# )

In [ ]:
import importlib
etl = importlib.import_module("scripts.etl_fixtures")
# etl.etl_parametrizable.serve(name="ETL-Fixtures", endpoints=["fixtures"])

## 5. Monitoreo en Prefect

El monitoreo del flujo no se realiza desde el notebook, sino desde la **UI de Prefect Cloud**.  
Una vez que el flow se ejecuta (corrida única o servido como agente), es posible:

1. **Flows** → ver el listado de flows registrados (ejemplo: `ETL-Fixtures`).  
2. **Run history** → revisar el historial de ejecuciones con sus logs, tiempos y reintentos.  
3. **Blocks** → administrar la configuración de almacenamiento o infraestructura remota si se utiliza.  
4. **Schedules** → programar ejecuciones automáticas (requiere plan pago).
